# Applied Data Science Capstone Project
#### Capstone Project for IBM Certification as a Data Scientist Professional

In [1]:
import pandas as pd
import numpy as np

#Import additional Libraries needed
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Find and Import Data

Retrieved data from these two sources:<br>
<br>
    https://gist.github.com/erichurst/7882666 <br>
    https://www.tripsavvy.com/cleveland-area-zip-codes-753059<br>

Then combined in excel with vlookup for the Greater Cleveland area. 

In [2]:
#Read data from csv
gd=pd.read_csv("cledata.csv")

In [3]:
gd = gd.groupby(['Zipcode','Latitude','Longitude'])['City'].apply(', '.join).reset_index()

In [90]:
gd.head(10)

,Zipcode,Latitude,Longitude,City
0,44004,41.855940,-80.791866,Ashtabula
1,44011,41.445100,-82.005109,Avon Lake
2,44012,41.498342,-82.017368,Avon
3,44017,41.370548,-81.861757,Berea
4,44021,41.443260,-81.144465,Burton
5,44022,41.446269,-81.402969,"Chagrin Falls , Moreland Hills"
6,44023,41.384751,-81.285759,Bainbridge
7,44024,41.577765,-81.192433,Chardon
8,44026,41.528147,-81.324706,Chesterland
9,44039,41.386000,-82.024711,North Ridgeville


In [88]:
indy=gd.iloc[46]
indy.to_frame().transpose()

,Zipcode,Latitude,Longitude,City
46,44131,41.3828,-81.6502,"Independence , Seven Hills"


In [5]:
gd.shape

(66, 4)

## Map Data

In [6]:
# create map of Toronto using latitude and longitude values
map_cle = folium.Map(location=[41.4993, -81.6944], zoom_start=10)

# add markers to map
for lat, lng, cty, zipcd in zip(gd['Latitude'], gd['Longitude'], gd['City'], gd['Zipcode']):
    label = '{}, {}'.format(cty, zipcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cle)  
    
map_cle

In [7]:
#Add Foursquare Credentials
CLIENT_ID = 'DRBPBLKZBJZOGBX1AGINJD1LJQYFJGITXQI2QI2XXUPFAI0W' # your Foursquare ID
CLIENT_SECRET = 'U0DJSYFII1DOA5KY1ZYOUPQIVZBN3CV5QYMRAXMOZ2X0XNUP' # your Foursquare Secret
VERSION = '20180605'

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials:
CLIENT_ID: DRBPBLKZBJZOGBX1AGINJD1LJQYFJGITXQI2QI2XXUPFAI0W
CLIENT_SECRET:U0DJSYFII1DOA5KY1ZYOUPQIVZBN3CV5QYMRAXMOZ2X0XNUP


In [39]:
gd.loc[46, 'City']

'Independence , Seven Hills '

In [41]:
neighborhood_latitude = gd.loc[46, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = gd.loc[46, 'Longitude'] # neighborhood longitude value

neighborhood_name = gd.loc[46, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Independence , Seven Hills  are 41.382841, -81.650195.


In [46]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 8000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=DRBPBLKZBJZOGBX1AGINJD1LJQYFJGITXQI2QI2XXUPFAI0W&client_secret=U0DJSYFII1DOA5KY1ZYOUPQIVZBN3CV5QYMRAXMOZ2X0XNUP&v=20180605&ll=41.382841,-81.650195&radius=8000&limit=100'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd530f971c428001be085cc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Independence',
  'headerFullLocation': 'Independence',
  'headerLocationGranularity': 'city',
  'totalResults': 229,
  'suggestedBounds': {'ne': {'lat': 41.45484107200007,
    'lng': -81.5544136201213},
   'sw': {'lat': 41.31084092799993, 'lng': -81.7459763798787}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e6821681fc75c9202996517',
       'name': 'Melt Bar & Grilled',
       'location': {'address': '6700 Rockside Rd',
        'lat': 41.39405650701098,
        'lng': -81.64434413320464,
        'labeledLatLngs': [{'label': '

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Melt Bar & Grilled,Sandwich Place,41.394057,-81.644344
1,Aladdin's Eatery,Mediterranean Restaurant,41.395764,-81.641843
2,Delmonico's Steakhouse,Steakhouse,41.394970,-81.648682
3,LockKeepers,American Restaurant,41.393972,-81.628454
4,LongHorn Steakhouse,Steakhouse,41.400290,-81.662691


In [64]:
categoriesgrouped=nearby_venues.groupby('categories').count()
categoriesgrouped.head()

,name,lat,lng
categories,,,
American Restaurant,7,7,7
Asian Restaurant,1,1,1
BBQ Joint,1,1,1
Bakery,4,4,4
Breakfast Spot,1,1,1


In [69]:
categoriesgrouped.sort_values('name',ascending=False).head()

,name,lat,lng
categories,,,
American Restaurant,7,7,7
Sandwich Place,7,7,7
Coffee Shop,6,6,6
Park,6,6,6
Bakery,4,4,4


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=8000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
#ash_venues = getNearbyVenues(names=gd['City'],latitudes=gd['Latitude'],longitudes=gd['Longitude'])


In [109]:
print(ash_venues.shape)
ash_venues

(475, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ashtabula,41.855940,-80.791866,Domino's Pizza,41.854736,-80.786767,Pizza Place
1,Ashtabula,41.855940,-80.791866,Circle K,41.854803,-80.786620,Convenience Store
2,Ashtabula,41.855940,-80.791866,Fay's Coffee Kitchen,41.855420,-80.788670,Coffee Shop
3,Avon,41.498342,-82.017368,Las Cazuelas,41.494242,-82.017597,Mexican Restaurant
4,Avon,41.498342,-82.017368,The Landing,41.495411,-82.017943,Plaza
5,Avon,41.498342,-82.017368,Romeo's Pizza,41.495279,-82.021231,Pizza Place
6,Avon,41.498342,-82.017368,Neon beach,41.495279,-82.021231,Spa
7,Avon,41.498342,-82.017368,Erie Burger Company,41.494867,-82.020644,Burger Joint
8,Avon,41.498342,-82.017368,The Landings,41.493943,-82.017721,Shopping Mall
9,Berea,41.370548,-81.861757,PNC Bank,41.371800,-81.865570,Bank


In [102]:
ash_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ashtabula,3,3,3,3,3,3
Aurora,4,4,4,4,4,4
Avon,6,6,6,6,6,6
Bay Village,7,7,7,7,7,7
"Beachwood , Shaker Heights",1,1,1,1,1,1


In [103]:
print('There are {} uniques categories.'.format(len(ash_venues['Venue Category'].unique())))

There are 166 uniques categories.


In [20]:
ash_venues['Venue Category'].unique()

array(['Pizza Place', 'Convenience Store', 'Coffee Shop',
       'Mexican Restaurant', 'Plaza', 'Spa', 'Burger Joint',
       'Shopping Mall', 'Bank', 'Liquor Store', 'Pharmacy',
       'Fast Food Restaurant', 'Optical Shop', 'Steakhouse', 'Pet Store',
       'Electronics Store', 'Baseball Field', 'Trail',
       'Italian Restaurant', 'Gym', 'Sandwich Place', 'Market',
       'Automotive Shop', 'Sushi Restaurant',
       'Construction & Landscaping', 'Salon / Barbershop', 'Wine Shop',
       'Greek Restaurant', 'Sporting Goods Shop', 'Business Service',
       'American Restaurant', 'Asian Restaurant', 'Sports Bar',
       'BBQ Joint', "Men's Store", 'Music Store', 'Big Box Store',
       'Donut Shop', 'Fabric Shop', 'Weight Loss Center', 'Bar',
       'Supplement Shop', 'Chocolate Shop', 'Hardware Store',
       'Mobile Phone Shop', 'Breakfast Spot', 'Indian Restaurant',
       'Grocery Store', 'Gas Station', 'Martial Arts Dojo',
       'Athletics & Sports', 'Diner', 'Supermarket', 'V

In [104]:
# one hot encoding
ash_onehot = pd.get_dummies(ash_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ash_onehot['Neighborhood'] = ash_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ash_onehot.columns[-1]] + list(ash_onehot.columns[:-1])
ash_onehot = ash_onehot[fixed_columns]

ash_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,...,Toy / Game Store,Trail,Video Game Store,Video Store,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Ashtabula,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ashtabula,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashtabula,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Avon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Avon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
ash_onehot.shape

(475, 167)

In [95]:
ash_grouped = ash_onehot.groupby('Neighborhood').mean().reset_index()
ash_grouped.head(50)

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,...,Toy / Game Store,Trail,Video Game Store,Video Store,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Ashtabula,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Aurora,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Avon,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bay Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Beachwood , Shaker Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bedford , Oakwood Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Berea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Broadview Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Brooklyn,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Brookpark,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
# paste info here

In [25]:
ash_grouped.shape

(50, 167)

In [26]:
ash_grouped.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,...,Toy / Game Store,Trail,Video Game Store,Video Store,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Ashtabula,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aurora,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Avon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bay Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Beachwood , Shaker Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
num_top_venues = 5

for hood in ash_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ash_grouped[ash_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ashtabula ----
               venue  freq
0        Coffee Shop  0.33
1        Pizza Place  0.33
2  Convenience Store  0.33
3  Accessories Store  0.00
4    Organic Grocery  0.00


----Aurora ----
                     venue  freq
0     Gym / Fitness Center  0.25
1  New American Restaurant  0.25
2                BBQ Joint  0.25
3              Golf Course  0.25
4        Accessories Store  0.00


----Avon ----
                venue  freq
0               Plaza  0.17
1                 Spa  0.17
2         Pizza Place  0.17
3        Burger Joint  0.17
4  Mexican Restaurant  0.17


----Bay Village ----
          venue  freq
0          Park  0.43
1       Theater  0.14
2         Beach  0.14
3  Soccer Field  0.14
4  Dessert Shop  0.14


----Beachwood , Shaker Heights ----
                       venue  freq
0           Department Store   1.0
1          Accessories Store   0.0
2            Organic Grocery   0.0
3  Middle Eastern Restaurant   0.0
4         Miscellaneous Shop   0.0


----Bedford , 

                    venue  freq
0  Furniture / Home Store  0.07
1           Grocery Store  0.05
2                    Bank  0.05
3      Salon / Barbershop  0.04
4          Discount Store  0.04


----Perry ----
                       venue  freq
0         Athletics & Sports  0.67
1           Business Service  0.33
2          Accessories Store  0.00
3            Organic Grocery  0.00
4  Middle Eastern Restaurant  0.00


----Rocky River ----
              venue  freq
0              Food  0.14
1      Intersection  0.14
2        Playground  0.14
3  Football Stadium  0.14
4    Baseball Field  0.14


----Solon ----
            venue  freq
0     Pizza Place  0.10
1            Bank  0.10
2  Sandwich Place  0.10
3     Flower Shop  0.05
4    Burger Joint  0.05


----Strongsville ----
                  venue  freq
0  Gym / Fitness Center  0.14
1        Cosmetics Shop  0.07
2           Gas Station  0.07
3          Intersection  0.07
4        Sandwich Place  0.07


----Twinsburg ----
                

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ash_grouped['Neighborhood']

for ind in np.arange(ash_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ash_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ashtabula,Convenience Store,Coffee Shop,Pizza Place,Yoga Studio,Dessert Shop
1,Aurora,Gym / Fitness Center,Golf Course,BBQ Joint,New American Restaurant,Yoga Studio
2,Avon,Plaza,Pizza Place,Mexican Restaurant,Burger Joint,Shopping Mall
3,Bay Village,Park,Theater,Dessert Shop,Beach,Soccer Field
4,"Beachwood , Shaker Heights",Department Store,Yoga Studio,Diner,Fabric Shop,Entertainment Service


In [30]:
# set number of clusters
kclusters = 4

ash_grouped_clustering = ash_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ash_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 2, 2, 2, 2, 2])

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cle_merge = gd

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cle_merge = cle_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City', how = 'right')
cle_merge.head() 

,Zipcode,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,44004,41.855940,-80.791866,Ashtabula,2,Convenience Store,Coffee Shop,Pizza Place,Yoga Studio,Dessert Shop
2,44012,41.498342,-82.017368,Avon,2,Plaza,Pizza Place,Mexican Restaurant,Burger Joint,Shopping Mall
3,44017,41.370548,-81.861757,Berea,2,Liquor Store,Bank,Pharmacy,Pet Store,Steakhouse
5,44022,41.446269,-81.402969,"Chagrin Falls , Moreland Hills",2,Electronics Store,Yoga Studio,Farmers Market,Fabric Shop,Entertainment Service
7,44024,41.577765,-81.192433,Chardon,2,Baseball Field,Trail,Yoga Studio,Diner,Fabric Shop


In [32]:
cle_merge

,Zipcode,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,44004,41.855940,-80.791866,Ashtabula,2,Convenience Store,Coffee Shop,Pizza Place,Yoga Studio,Dessert Shop
2,44012,41.498342,-82.017368,Avon,2,Plaza,Pizza Place,Mexican Restaurant,Burger Joint,Shopping Mall
3,44017,41.370548,-81.861757,Berea,2,Liquor Store,Bank,Pharmacy,Pet Store,Steakhouse
5,44022,41.446269,-81.402969,"Chagrin Falls , Moreland Hills",2,Electronics Store,Yoga Studio,Farmers Market,Fabric Shop,Entertainment Service
7,44024,41.577765,-81.192433,Chardon,2,Baseball Field,Trail,Yoga Studio,Diner,Fabric Shop
9,44039,41.386000,-82.024711,North Ridgeville,3,Pet Store,Yoga Studio,Dessert Shop,Entertainment Service,Electronics Store
11,44041,41.776900,-80.949898,"Geneva , Geneva on the Lake",2,Convenience Store,Yoga Studio,Diner,Fabric Shop,Entertainment Service
13,44056,41.314919,-81.501633,Macedonia,2,Sandwich Place,Gym,Automotive Shop,Market,Italian Restaurant
14,44057,41.760117,-81.060305,Madison,0,Construction & Landscaping,Farmers Market,Fabric Shop,Entertainment Service,Electronics Store
15,44060,41.676530,-81.328167,Mentor,2,Fast Food Restaurant,Sporting Goods Shop,Wine Shop,Greek Restaurant,Burger Joint


In [33]:
# create map
map_clusters = folium.Map(location=[41.4993, -81.6944], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cle_merge['Latitude'], cle_merge['Longitude'], cle_merge['City'], cle_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [97]:
Cluster0=cle_merge.loc[cle_merge['Cluster Labels'] == 0, cle_merge.columns[[3] + list(range(5, cle_merge.shape[1]))]]
Cluster0

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Madison,Construction & Landscaping,Farmers Market,Fabric Shop,Entertainment Service,Electronics Store
19,Perry,Athletics & Sports,Business Service,Yoga Studio,Diner,Fabric Shop
21,"Wickliffe , Willowick",Golf Course,Park,Yoga Studio,Dessert Shop,Entertainment Service
39,"Cleveland Heights , South Euclid",Park,Rental Car Location,Yoga Studio,Dessert Shop,Entertainment Service
46,"Independence , Seven Hills",Coffee Shop,Park,Baseball Field,Yoga Studio,Diner
48,Parma,Campground,Park,Business Service,Construction & Landscaping,Diner
53,Bay Village,Park,Theater,Dessert Shop,Beach,Soccer Field


In [116]:
Cluster1=cle_merge.loc[cle_merge['Cluster Labels'] == 1, cle_merge.columns[[3] + list(range(5, cle_merge.shape[1]))]]
Cluster1

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
29,"Cleveland Old Brooklyn , Cleveland Tremont",Playground,Yoga Studio,Dessert Shop,Entertainment Service,Electronics Store


In [117]:
Cluster2=cle_merge.loc[cle_merge['Cluster Labels'] == 2, cle_merge.columns[[3] + list(range(5, cle_merge.shape[1]))]]
Cluster2

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ashtabula,Convenience Store,Coffee Shop,Pizza Place,Yoga Studio,Dessert Shop
2,Avon,Plaza,Pizza Place,Mexican Restaurant,Burger Joint,Shopping Mall
3,Berea,Liquor Store,Bank,Pharmacy,Pet Store,Steakhouse
5,"Chagrin Falls , Moreland Hills",Electronics Store,Yoga Studio,Farmers Market,Fabric Shop,Entertainment Service
7,Chardon,Baseball Field,Trail,Yoga Studio,Diner,Fabric Shop
11,"Geneva , Geneva on the Lake",Convenience Store,Yoga Studio,Diner,Fabric Shop,Entertainment Service
13,Macedonia,Sandwich Place,Gym,Automotive Shop,Market,Italian Restaurant
15,Mentor,Fast Food Restaurant,Sporting Goods Shop,Wine Shop,Greek Restaurant,Burger Joint
17,North Olmsted,American Restaurant,Sandwich Place,Fast Food Restaurant,Coffee Shop,Pizza Place
20,Twinsburg,Pizza Place,Bank,Shipping Store,Plaza,Coffee Shop


In [118]:
Cluster3=cle_merge.loc[cle_merge['Cluster Labels'] == 3, cle_merge.columns[[3] + list(range(5, cle_merge.shape[1]))]]
Cluster3

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,North Ridgeville,Pet Store,Yoga Studio,Dessert Shop,Entertainment Service,Electronics Store


In [119]:
Cluster4=cle_merge.loc[cle_merge['Cluster Labels'] == 4, cle_merge.columns[[3] + list(range(5, cle_merge.shape[1]))]]
Cluster4

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
